# Inclusion of scripts in SoS workflows

* **Difficulty level**: easy
* **Time need to lean**: 10 minutes or less
* **Key points**:
  * A Python function with a string as the first parameter can be written in a script style
  * Scripts are included in verbatim by default
  * Indentation is not required but recommended
  * `expand=True` turns the script into a Python f-string
  * `expand="l r"` can be used to specify another delimiter for string interpolation

## SoS "Actions"

SoS defines a number of **actions**, which are simply Python functions that follows a specific set of conventions. For example, an action `sh` is a python function that executes its first parameter as a shell script:

In [1]:
sh('echo Hello world')

Hello world


When you execute a SoS script from command line or SoS Notebook, these functions are automatically imported and can be used directly, and you can use them just like any other Python functions. For example,

In [3]:
greeting = "Hello world"
sh(f'echo {greeting}')

Hello world


Here we define a Python string `greeting` and use the `sh` action to execute a shell script. A [Python f-string](https://www.python.org/dev/peps/pep-0498/) is used to compose the script with the defined variable.

<div class="bs-callout bs-callout-info" role="alert">
  <h4>Python f-string</h4>
  <p>SoS uses Python f-string extensively. Please read <a href="https://www.python.org/dev/peps/pep-0498/">PEP498</a> or one of the online tutorials on how to use Python f-strings if you are not familiar with it.</p>  
</div>

When the scripts get longer, you will have to use Python multi-line strings to pass them to the action. Here is an example for the use of a `R` action to execute a longer `R` script:

In [5]:
R("""\
x <- 5
if(x >= 0) {
    print("Non-negative number")
} else {
    print("Negative number")
}
""")

[1] "Non-negative number"


The use of format string in these cases become more complicated. First, you will need to use multi-line f-string (`f'''x'''` and `f"""x"""`). Second, when the script itself contains braces, they will be interpreted as Python expressions unless they are doubled.

Consequently, a `R` script that uses a Python variable `my_num` needs to be written as follows:

In [6]:
my_num = -1

R(f"""\
x <- {my_num}
if(x >= 0) {{
    print("Non-negative number")
}} else {{
    print("Negative number")
}}
""")

[1] "Negative number"


## Script style function calls <a id="Script_style_function_call"></a>

The f-string in the last example is not quite readable, error-prone, and difficult to maintaine, especially when the script contains multiple braces and variables. For this reason, SoS introduces a special syntax that allows you to write Python functions that accept a script (string) as the first parameter in a special script format.

For example,

In [7]:
R("""\
x <- 5
if(x >= 0) {
    print("Non-negative number")
} else {
    print("Negative number")
}
""")

[1] "Non-negative number"


can be written as

In [9]:
R:
x <- 5
if(x >= 0) {
    print("Non-negative number")
} else {
    print("Negative number")
}

[1] "Non-negative number"


or better as

In [8]:
R:
    x <- 5
    if(x >= 0) {
        print("Non-negative number")
    } else {
        print("Negative number")
    }

[1] "Non-negative number"


Indentation is optional and SoS will automatically dedent the included scripts.

<div class="bs-callout bs-callout-info" role="alert">
  <h4>Indentation of scripts</h4>
  <p>The indentation of scripts in the script style is optional but recommended, because it allows the separation of multiple function calls such as</p>
    <pre>
    R:
       cat('this is R')
       
    python:
       print('this is python')
    </pre>
</div>

### The `expand` option

When option `expand=True` is specified, the included script will be treated as a Python f-string. For example,

In [ ]:
my_num = -1

R(f"""\
x <- {my_num}
if(x >= 0) {{
    print("Non-negative number")
}} else {{
    print("Negative number")
}}
""")

can be written as

In [10]:
my_num = -1

R: expand=True
    x <- {my_num}
    if(x >= 0) {{
        print("Non-negative number")
    }} else {{
        print("Negative number")
    }}

[1] "Negative number"


Note that SoS Notebook automatically highlights the interpolated parts of the included script, which makes it much easier to differentiate Python expressions from the original R script.

Because the included script has two pairs of braces, it is necessary to double them so that they are not treated as Python expressions. In these cases, it is actually easier to use a different set of sigil (delimiters) for string interpolation. This can be done using the `expand` option as follows:

In [11]:
my_num = -1

R: expand='${ }'
    x <- ${my_num}
    if(x >= 0) {
        print("Non-negative number")
    } else {
        print("Negative number")
    }

[1] "Negative number"


The sigil should be specified as a string with left and right sigil separated by a space. You can use any pair of sigils as long as they do not cause confusion. 

<div class="bs-callout bs-callout-info" role="alert">
  <h4>Use of alternative sigils</h4>
  <p>When the included scripts have braces, it is easier to use a set of alternative sigils for string interpolation.</p>
</div>

## Further reading
* [SoS Actions and common action options](doc/user_guide/sos_actions.html)